# Notebook 1:  The FHIR API

## Learning the FHIR REST API

In [ ]:
# setup imports
import os
from requests import get
from requests import post
from requests import put
from requests import delete
from requests import head

# save the base url of the server
base = 'https://cluster1-573846-250babbbe4c3000e15508cd07c1d282b-0000.us-east.containers.appdomain.cloud/open'

# define some useful functions
def peek(string, line_count=10):
    print(os.linesep.join(string.split(os.linesep)[:line_count]) + '\n')

In [ ]:
#(Optional)

# install jsonpointer
!pip install jsonpointer

from jsonpointer import resolve_pointer as resolve

In [ ]:
# retrieve the server "CapabilityStatement" and print the important bits
response = get(base + '/metadata')
print('Response code: ' + str(response.status_code))
result = response.json()
print('Server: ' + result['name'] + ' ' + result['version'])
print('Security: ' + str(result['rest'][0]['security']['service']))
resources = result['rest'][0]['resource']

supported_types = {r['type']: [i['code'] for i in r['interaction']] for r in resources}

print('Supported types: ')
for k,v in supported_types.items():
    print('  ' + k + ': ' + str(v))

In [ ]:
# the type of resource we want to query
type = 'Patient'

# retrieve all resources of a given type and print the HTTP status code and the first 20 lines of the response
response = get(base + '/' + type)
print('Response code: ' + str(response.status_code))
peek('Response body: \n' + response.text, 20)

# technically you've now performed your first FHIR "search" (just with no parameters)

In [ ]:
# results are paged and the "link" field in the response Bundle contains links to previous, current, and next page of results
for link in response.json().get('link'):
    if link.get('relation') == 'next':
        page2 = get(link.get('url'))        
peek('Second page: \n' + page2.text, 20)
print('Number of entries: ' + str(len(response.json().get('entry'))))

In [ ]:
# we can control the number of search results by passing the _count parameter
response = get(base + '/Patient?_count=1')
peek('Single resource per page: \n' + response.text, 25)
print('Number of entries: ' + str(len(response.json().get('entry'))))

In [ ]:
# if you're only interested in the count, you can specify that via either
# A. _count=0 (0 results per page); or
# B. _summary=count

print(get(base + '/Patient' + '?' + '_summary=count').text)

In [ ]:
# if you want a lot of results per page, you can reduce the amount of data returned via the _summary or _elements parameters

# https://www.hl7.org/fhir/search.html#summary
# look for the Σ flag in the Resource Content section of the resource page in the specification for what elements are considered "summary" elements 
reponse = get(base + '/Patient?_count=1' + '&' + '_summary=true')
print('Summary: \n' + str(response.json().get('entry')[0].get('resource').keys()))

In [ ]:
# need more control?
# you can use the _elements parameter to ask for specific fields back (although the server should include required fields and modifier fields as well)
response = get(base + '/Patient?_count=1' + '&' + '_elements=id,gender')
print('Elements: \n' + str(response.json().get('entry')[0].get('resource').keys()))

In [ ]:
# this can add up!

response = get(base + '/Patient?_count=100')
print('Normal: \t' + str(len(response.content)) + ' bytes \t(' + str(response.elapsed.total_seconds()) + ' s)')

response = get(base + '/Patient?_count=100&_summary=true')
print('Summary: \t' + str(len(response.content)) + ' bytes \t(' + str(response.elapsed.total_seconds()) + ' s)')

response = get(base + '/Patient?_count=100&_elements=id,gender,birthDate')
print('Elements: \t' + str(len(response.content)) + ' bytes \t(' + str(response.elapsed.total_seconds()) + ' s)')


In [ ]:
# now add some search parameters

# each FHIR resource type has its own set of parameters; find them toward the bottom of the page for that resource type in the specification
# for example, for the Patient resource type, see https://www.hl7.org/fhir/patient.html#search
response = get(base + '/Patient' + '?' + 'gender=male')
print('Response code: ' + str(response.status_code))
peek('Response body: \n' + response.text, 25)

In [ ]:
print('male:   \t' + str(get(base + '/Patient' + '?' + 'gender=male' + '&' + '_summary=count').json().get('total')))
print('female: \t' + str(get(base + '/Patient' + '?' + 'gender=female' + '&' + '_summary=count').json().get('total')))

In [ ]:
# use the "missing" modifier to look for resources that do NOT have a value for the target parameter
response = get(base + '/Patient' + '?' + 'gender:missing=true' + '&' + '_summary=count')
print('missing gender: ' + str(response.json().get('total')))

In [ ]:
# search parameters have types

# gender is considered a "token" search parameter

# Token search
# this parameter type is common for 'coded' values (Code, Coding, and CodeableConcept) and identifiers
# token values consist of a system and a code, although sometimes the system is implicit (like in the case of gender)
# users can search on the system and code (system|code), the code alone (code), system-less codes (|code), or even the system alone (system|)
response = get(base + '/Patient' + '?' + 'gender=http://hl7.org/fhir/administrative-gender|male' + '&_count=1&_elements=gender')
peek('male:\n' + str(response.json()))


# there are also Number, Date/DateTime, String, Reference, Quantity, URI, and Composite parameter types

In [ ]:
# String search
response = get(base + '/Patient' + '?' + 'family=Smith' + '&_elements=name')
print('Smiths:')
for entry in response.json().get('entry'):
    resource = entry.get('resource')
    print(resource.get('id'), end=': ')
    print(', '.join(map(lambda n: n.get('family'), resource.get('name'))))
        

In [ ]:
# wait, "Smitham" !?

# string search performs a case-insensitive "begins-with" search by default!
# use the modifier ":exact" if you want exact matches (and improved performance)
response = get(base + '/Patient' + '?' + 'family:exact=Smith' + '&_elements=name')
print('Smiths:')
for entry in response.json().get('entry'):
    resource = entry.get('resource')
    print(resource.get('id'), end=': ')
    print(', '.join(map(lambda n: n.get('family'), resource.get('name'))))
print()

# string search also has a ":contains" modifier
response = get(base + '/Patient' + '?' + 'family:contains=ski' + '&_elements=name')
print('Skis:')
for entry in response.json().get('entry'):
    resource = entry.get('resource')
    print(resource.get('id'), end=': ')
    print(', '.join(map(lambda n: n.get('family'), resource.get('name'))))


In [ ]:
# Date search
response = get(base + '/Patient' + '?' + 'birthdate=1984' + '&_elements=birthDate')
print('Born in 1984:')
for entry in response.json().get('entry'):
    resource = entry.get('resource')
    print(resource.get('id'), end=': ')
    print(resource.get('birthDate'))

In [ ]:
# date searches support lt(<), le(<=), gt(>), ge(>=), sa(starts after), and eb(ends before) "prefixes"
response = get(base + '/Patient' + '?' + 'birthdate=eb1984' + '&_elements=birthDate')
print('Born before 1984:')
for entry in response.json().get('entry'):
    resource = entry.get('resource')
    print(resource.get('id'), end=': ')
    print(resource.get('birthDate'))

response = get(base + '/Patient' + '?' + 'birthdate=sa1984' + '&_elements=birthDate')
print('\n' + 'Born after 1984:')
for entry in response.json().get('entry'):
    resource = entry.get('resource')
    print(resource.get('id'), end=': ')
    print(resource.get('birthDate'))

# some servers support ap(approximately equal) as well, although the spec lets the server decide exactly what that means...
response = get(base + '/Patient' + '?' + 'birthdate=ap1984' + '&_elements=birthDate')
print('\n' + 'Born "around" 1984:')
for entry in response.json().get('entry'):
    resource = entry.get('resource')
    print(resource.get('id'), end=': ')
    print(resource.get('birthDate'))

In [ ]:
# Reference search
response = get(base + '/Patient?general-practitioner:missing=false&_elements=generalPractitioner,link,managingOrganization&_count=1')
peek('Patients with a general-practitioner:   \n' + str(response.json()))

# since our model doesn't have any reference fields on the Patient resources, lets look at Observations instead
response = get(base + '/Condition' + '?' + 'subject=Patient/17598beef3c-73a65dab-c8e5-4756-a60a-69bbc48cef4f' + '&_elements=code')
print('Conditions for patient 17598beef3c-73a65dab-c8e5-4756-a60a-69bbc48cef4f:')
for entry in response.json().get('entry'):
    resource = entry.get('resource')
    print(resource.get('code'))

# when the type of the reference is fixed to a single value, it can be omitted (Patient/x -> x)
response2 = get(base + '/Condition' + '?' + 'patient=17598beef3c-73a65dab-c8e5-4756-a60a-69bbc48cef4f' + '&_elements=code')
print('\n' + 'Result entries match? ' + str(response.json().get('entry') == response2.json().get('entry')))

# a reference to a resource's full url on the server should be equivalent to the relative reference format mentioned above
response3 = get(base + '/Condition' + '?' + 'patient=' + base + '/Patient/17598beef3c-73a65dab-c8e5-4756-a60a-69bbc48cef4f' + '&_elements=code')
print('\n' + 'Result entries match? ' + str(response.json().get('entry') == response3.json().get('entry')))

# references can also reference resources on other servers

In [ ]:
# Chaining

# where reference parameters get really interesting is when you want to query one resource type based on a property of another resource to which its linked
# for example, here is a search for Type II Diabetes in female patients
response = get(base + '/Condition' + '?' + 'code=http://snomed.info/sct|44054006' + '&' + 'patient:Patient.gender=female' + '&_count=1')
peek('Type II Diabetes in female patients:   \n' + str(response.json()))
    
# for example, here is a search for blood pressure values for female patients
# TODO: fix this query!
#response = get(base + '/Observation' + '?' + 'code=http://loinc.org|85354-9' + '&' + 'date=gt2020-01-01' + '&' + 'patient:Patient.gender=female' + '&_count=1')
#peek('Blood Pressure Observations for female patients since 2020-01-01:   \n' + str(response.json()))

In [ ]:
# Reverse chaining

# references can be search the other way around via the "_has" parameter
response = get(base + '/Patient?gender=female' + '&' + '_has:Condition:patient:code=http://snomed.info/sct|44054006' + '&_count=1')
peek('Female patients with Type II Diabetes:   \n' + str(response.json()))

# TODO: fix this query!
#response = get(base + '/Patient?gender=female' + '&' + '_has:Observation:patient:code=http://loinc.org|85354-9' + '&_count=1')
#peek('Female patients with Blood Pressure Observations:   \n' + str(response.json()))

In [ ]:
# Includes

# its also possible to get a resource and its related resources back in a single query
response = get(base + '/Condition?code=http://snomed.info/sct|44054006' + '&' + '_include=Condition:patient' + '&' + 'patient:Patient.gender=female' + '&_count=2')
peek('Response contains both Conditions and Patients, but only the Conditions are counted in the page size and total:')
print('Total: ' + str(response.json().get('total')))
for entry in response.json().get('entry'):
    resource = entry.get('resource')
    print(resource.get('resourceType'), end=': ')
    print(resource.get('id'))


In [ ]:
# Reverse Includes

response = get(base + '/Patient?gender=female' + '&' + '_revinclude=Condition:patient' + '&_count=2')
peek('Response contains both Patients and Conditions, but only the Patients are counted in the page size and total:')
print('Total: ' + str(response.json().get('total')))
for entry in response.json().get('entry'):
    resource = entry.get('resource')
    print(resource.get('resourceType'), end=': ')
    print(resource.get('id'))

## Putting it together

In [ ]:
response = get(base + '/Condition' + '?' + 'code=http://snomed.info/sct|44054006' + '&_count=1')
print('Patients with Type II Diabetes:  ' + str(response.json().get('total')))

In [ ]:
# SNOMED concepts for comorbidities of Type II Diabetes
#coronary heart disease (CHD), 53741008
#chronic kidney disease (CKD), 709044004
#atrial fibrillation, 49436004
#stroke, 230690007
#hypertension, 38341003
#heart failure, 84114007
#peripheral vascular disease (PVD), 400047006
#rheumatoid arthritis, 69896004
#Malignant neoplasm, primary (morphologic abnormality), 86049000
#Malignant neoplastic disease (disorder), 363346000
#osteoporosis, 64859006
#depression, 35489007
#asthma, 195967001
#chronic obstructive pulmonary disease (COPD), 13645005
#dementia, 52448006
#severe mental illness (SMI), 391193001
#epilepsy, 84757009
#hypothyroidism, 40930008
#learning disability, 1855002

print('CHD: \t\t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '53741008').json().get('total')))
print('CKD: \t\t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '709044004').json().get('total')))
print('AFib: \t\t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '49436004').json().get('total')))
print('stroke: \t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '230690007').json().get('total')))
print('hypertension: \t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '38341003').json().get('total')))
print('heart failure: \t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '84114007').json().get('total')))
print('PVD: \t\t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '400047006').json().get('total')))
print('arthritis: \t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '69896004').json().get('total')))
print('cancer: \t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '86049000').json().get('total') + get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '363346000').json().get('total')))
print('osteoporosis: \t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '64859006').json().get('total')))
print('depression: \t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '35489007').json().get('total')))
print('asthma: \t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '195967001').json().get('total')))
print('COPD: \t\t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '13645005').json().get('total')))
print('dementia: \t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '52448006').json().get('total')))
print('SMI: \t\t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '391193001').json().get('total')))
print('epilepsy: \t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '84757009').json().get('total')))
print('hypothyroidism: \t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '40930008').json().get('total')))
print('learning disability: \t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '1855002').json().get('total')))

In [ ]:
# Patients with Type II Diabetes *and* comorbidities
hasDiabetes = base + '/Patient?_elements=id&_has:Condition:patient:code=http://snomed.info/sct|44054006'

def printPatientsWithComorbidity(conceptId):
    responseJSON = get(hasDiabetes + '&_has:Condition:patient:code=http://snomed.info/sct|' + conceptId).json()
    print('Total: ' + str(responseJSON.get('total')))
    if 'entry' in responseJSON:
        for entry in responseJSON.get('entry'):
            print(entry.get('resource').get('id'), end=", ")
    print()

print('CHD:')
printPatientsWithComorbidity('53741008')

print('AFib:')
printPatientsWithComorbidity('49436004')

print('stroke:')
printPatientsWithComorbidity('230690007')

print('heart failure:')
printPatientsWithComorbidity('84114007')

print('arthritis:')
printPatientsWithComorbidity('69896004')

print('osteoporosis:')
printPatientsWithComorbidity('64859006')

print('asthma:')
printPatientsWithComorbidity('195967001')

print('epilepsy:')
printPatientsWithComorbidity('84757009')

In [ ]:
peek('A1c:' + str(get(base + '/Observation?_elements=code&code=http://loinc.org|' + '4548-4').text), 50)


## Bulk Export

## Using the fhirpy pip module

FHIR has a rich library of open source clients and servers for almost every lanuage.
For python, there is a module called `fhirpy` which can be found at https://pypi.org/project/fhirpy/

In [ ]:
!pip install fhirpy

In [ ]:
from fhirpy import SyncFHIRClient

fhir = SyncFHIRClient(base)

In [ ]:
searchSet = fhir.resources('Practitioner')
searchSet.fetch()